In [1]:
# coding=utf-8
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re

### Parse all Russian universities

In [12]:
url = 'http://indicators.miccedu.ru/monitoring/index.php?m=vpo'
r = requests.get(url)
r.encoding = r.apparent_encoding
page = BS(r.text, 'html.parser')

columns = []
uni_id_list = []
uni = []
data = pd.DataFrame(data = [[0, 1, 2, 3, 4]],
                    columns = ['uni_id', 'index', 'desc', 'dim', 'measure'])

for state in page.select('p.MsoListParagraph a[href]'): # through regions
    region_url = host + str(year) + '/' + state['href']
    r = requests.get(region_url)
    r.encoding = r.apparent_encoding
    state_page = BS(r.content)
    
    for uni in state_page.select('.blockcontent tr td.inst a[href]'): # through unis
        uni_id = int(uni['href'][12:])
        uni.append(uni.get_text())
        uni_id_list.append(uni_id)
        uni_url = host + '_vpo/' + uni['href']
        r = requests.get(uni_url)
        r.encoding = r.apparent_encoding
        uni_page = BS(r.content)
                    
        for indicator in uni_page.select('table#analis_dop tr'): # through the last table
            fields = []
            td_num = len(indicator.find_all('td'))
            
            if td_num == 4: # skip headings and not full rows
                for td in indicator.select('td'):
                    fields.append(td.get_text())
                row = pd.DataFrame(data = [[uni_id, fields[0], fields[1], fields[2], fields[3]]],
                                    columns = ['uni_id', 'index', 'desc', 'dim', 'measure'])
                data = data.append(row)

data = data.iloc[3:, :]
data.set_index('uni_id', inplace = True)
data.to_excel('russian_uni.xlsx', index = False)

In [19]:
url = 'http://indicators.miccedu.ru/monitoring/_vpo/inst.php?id='

unis = pd.read_excel('Data/Russian_Universities_Initial.xlsx')

for index, uni in unis.iterrows():
    r = requests.get(url + str(uni.Uni_Id))
    r.encoding = r.apparent_encoding
    page = BS(r.text, 'html.parser')
    
    
    for indicator in page.select('table#analis_dop tr'): # through the last table
        fields = []
        td_num = len(indicator.find_all('td'))
            
        if td_num == 4: # skip headings and not full rows
            not_valid = False
            for ix, td in enumerate(indicator.select('td')):
                if re.search('№', td.get_text()) or (ix == 2 and re.search('да', td.get_text())):
                    not_valid = True
                    break
                else:
                    fields.append(td.get_text())
            
            if not_valid == False:
                unis.loc[index, str(fields[0])] = float(fields[3].replace(' ', '').replace(',', '.'))


    # working with outpus
    # there is an additional empty tbody before thead of the result table, so we cannot use BS
    # use regex instead
    Es = re.findall('(E\.\d)</td>', str(page))
    output_values = re.findall('right center no-repeat;\">(\d+(?:,\d+)?)<', str(page))
    income = re.findall('<span style=\"\">(\d+(?:,\d+)?)<', str(page))
    output_values.insert(4, income[0])
    
    for ix, val in enumerate(output_values):
        unis.loc[index, Es[ix]] = float(val.replace(',', '.'))
            
unis.to_excel('Data/Russian_Universities.xlsx', index = False)
unis.head()

,University,Uni_Id,Name,Region,THE,QS,CWUR,ARWU,1,2,...,53,54,55,56,E.1,E.2,E.3,E.4,E.5,E.8
0,MSU,1725,МГУ,Moscow,189,84,222.0,87,34838.0,83.67,...,67.76,26.92,41.92,55.98,83.81,979.35,7.50,3637.88,201.07,11.69
1,MIPT,161,МФТИ,Moscow,201-250,302,539.0,401-500,6483.0,94.56,...,38.52,46.33,4.33,78.43,94.56,4061.84,11.00,8767.60,244.94,6.70
2,SSAU,251,СНИУ,Samara,1001+,651-700,NaN,NaN,14283.0,71.62,...,54.28,21.42,24.84,65.31,71.93,671.10,6.47,3205.62,246.01,5.10
3,MSTU,147,МГТУ,Moscow,801-1000,284,NaN,NaN,21728.0,79.07,...,63.31,25.64,13.70,40.16,79.07,1575.16,5.13,6349.84,209.63,5.23
4,HSE,1766,ВШЭ,Moscow,251-300,322,879.0,901-1000,28755.0,86.27,...,57.03,20.10,38.83,43.95,86.27,1461.66,9.56,8169.24,250.37,4.27


### Parse Desired Russian Universities

In [20]:
host = 'http://indicators.miccedu.ru/monitoring/'
year = 2018
query = '/_vpo/inst.php?id='
url = host + str(year) + query

unis = pd.read_excel('Data/Russian_Universities_Initial.xlsx')

for index, uni in unis.iterrows():
    r = requests.get(url + str(uni.Uni_Id))
    r.encoding = r.apparent_encoding
    page = BS(r.text, 'html.parser')
    
    
    for indicator in page.select('table#analis_dop tr'): # through the last table
        fields = []
        td_num = len(indicator.find_all('td'))
            
        if td_num == 4: # skip headings and not full rows
            not_valid = False
            for ix, td in enumerate(indicator.select('td')):
                if re.search('№', td.get_text()) or (ix == 2 and re.search('да', td.get_text())):
                    not_valid = True
                    break
                else:
                    fields.append(td.get_text())
            
            if not_valid == False:
                unis.loc[index, str(fields[0])] = float(fields[3].replace(' ', '').replace(',', '.'))


    # working with outpus
    # there is an additional empty tbody before thead of the result table, so we cannot use BS
    # use regex instead
    Es = re.findall('(E\.\d)<\/td>', str(page))
    output_values = re.findall('right center no-repeat;\">(\d+(?:,\d+)?)<', str(page))
    income = re.findall('<span style=\"\">(\d+(?:,\d+)?)<', str(page))
    output_values.insert(4, income[0])
    
    for ix, val in enumerate(output_values):
        unis.loc[index, Es[ix]] = float(val.replace(',', '.'))
            
unis.to_excel('Data/Russian_Universities.xlsx', index = False)
unis.head()

,University,Uni_Id,Name,Region,THE,QS,CWUR,ARWU,1,2,...,54,55,56,E.1,E.2,E.3,E.4,E.5,E.6,E.8
0,MSU,1725,МГУ,Moscow,189,84,222.0,87,34613.0,81.86,...,23.88,44.92,51.00,81.97,688.32,7.31,2882.57,184.48,65.0,11.66
1,MIPT,161,МФТИ,Moscow,201-250,302,539.0,401-500,6240.0,93.43,...,33.83,6.46,67.94,93.43,2263.35,9.63,6964.30,224.29,55.0,7.27
2,SSAU,251,СНИУ,Samara,1001+,651-700,NaN,NaN,14381.0,72.40,...,26.74,27.55,30.75,72.65,714.93,5.67,2673.47,241.07,75.0,5.63
3,MSTU,147,МГТУ,Moscow,801-1000,284,NaN,NaN,21021.0,77.78,...,32.91,12.72,60.43,77.78,1817.12,5.01,5522.70,201.58,85.0,6.43
4,HSE,1766,ВШЭ,Moscow,251-300,322,879.0,901-1000,25046.0,85.44,...,19.54,35.89,49.38,85.44,1189.38,8.85,7208.84,237.09,75.0,4.56


### Parse Shanghai Ranking

In [26]:
url = 'http://www.shanghairanking.com/ARWU2019.html'
r = requests.get(url)
page = BS(r.text, 'html.parser')

data = pd.DataFrame(data = [[0, 1, 2, 3, 4, 5, 6, 7]],
                    columns = ['Rank', 'Uni', 'Alumni', 'Award', 'HiCi', 'N&S', 'PUB', 'PCP'])

for i, uni in enumerate(page.select('table#UniversityRanking tr')):
    if i == 0:
        continue
    else:
        fields = []
        for j, indicator in enumerate(uni.select('td')):
            if j in [0, 1, 5, 6, 7, 8, 9, 10]:
                if j == 1:
                    ind = (indicator.select('a'))[0].text
                else:
                    ind = indicator.text
                fields.append(ind)
        row = pd.DataFrame(data = [[fields[0], fields[1], fields[2], fields[3], fields[4], fields[5], fields[6], fields[7]]],
                                    columns = ['Rank', 'Uni', 'Alumni', 'Award', 'HiCi', 'N&S', 'PUB', 'PCP'])
        data = data.append(row)

data.set_index('Rank', inplace = True)
data.to_excel('ARWU_Ranking_Full.xlsx')